## Introductory Stuff LeNet Tutorial

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        #print(x.shape)
        x = self.pool(F.relu(self.conv1(x)))
        #print(x.shape)
        x = self.pool(F.relu(self.conv2(x)))
        #print(x.shape)
        x = x.view(-1, 16 * 5 * 5)
        #print(x.shape)
        x = F.relu(self.fc1(x))
        #print(x.shape)
        x = F.relu(self.fc2(x))
        #print(x.shape)
        x = self.fc3(x)
        #print(x.shape)
        #1/0
        return x

In [4]:
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [5]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
            
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

[1,  2000] loss: 2.185
[1,  4000] loss: 1.801
[1,  6000] loss: 1.610
[1,  8000] loss: 1.533
[1, 10000] loss: 1.463
[1, 12000] loss: 1.424
[2,  2000] loss: 1.338
[2,  4000] loss: 1.323
[2,  6000] loss: 1.312
[2,  8000] loss: 1.267
[2, 10000] loss: 1.278
[2, 12000] loss: 1.262


In [6]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of plane : 77 %
Accuracy of   car : 67 %
Accuracy of  bird : 53 %
Accuracy of   cat : 25 %
Accuracy of  deer : 43 %
Accuracy of   dog : 39 %
Accuracy of  frog : 73 %
Accuracy of horse : 64 %
Accuracy of  ship : 59 %
Accuracy of truck : 59 %


## Use EfficientNet

In [17]:
model.eval()
with torch.no_grad():
    for i in testloader:
        out = np.round(model(i[0]),3).numpy()
        print(i[1])
        break

tensor([3, 8, 8, 0])


In [7]:
class Depth_sc(nn.Module):
    def __init__(self,nin,nout,k):
        super(Depth_sc, self).__init__()
        self.depth = nn.Conv2d(nin, nin*8, (1,k),groups=nin)
        self.point = nn.Conv2d(nin*8, nout, (k,1))
        self.act = nn.ReLU6()

    def forward(self, x):
        #x = np.min(F.relu(self.point(self.depth(x))),6)
        x = self.act(self.point(self.depth(x)))
        return x
    
class OrdConv(nn.Module):
    def __init__(self,nin,nout,k):
        super(OrdConv, self).__init__()
        self.layer = nn.Conv2d(nin, nout, k)
        self.act = nn.ReLU6()

    def forward(self, x):
        x = self.act(self.layer(x))
        return x
    
'''class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        #self.stage1 = nn.Conv2d(3, 8, 3) #32 to 16
        self.stage2 = Depth_sc(3,8,3) #16 to 8
        self.stage3 = Depth_sc(8,32,5) #8 to 4
        self.pool = nn.MaxPool2d(2, 2)
        self.fc = nn.Linear(5408,10)
        #self.bn1 = nn.BatchNorm2d(8)
        self.bn2 = nn.BatchNorm2d(8)
        #self.bn3 = nn.BatchNorm2d(64)
        self.out = nn.Softmax(1)
        
    def forward(self, x):
        #print(x.shape)
        #x = self.bn1(self.stage1(x))
        #print(x.shape)
        x = self.pool(self.stage2(x))
        #print(x.shape)
        #x = self.bn3(self.stage3(x))
        x = self.pool(self.stage3(x))
        #print(x.shape)
        #x = self.pool(x)
        #print(x.shape)
        x = x.view(-1,5408)
        #print(x.shape)
        x = self.fc(x)
        #print(x.shape)
        x = self.out(x)
        #print(x.shape)
        #print(x)
        #1/0
        return x'''

class Net_SC(nn.Module):
    def __init__(self):
        super(Net_SC,self).__init__()
        self.l1 = OrdConv(3,16,3)
        self.l2 = OrdConv(16,32,5)
        self.pool =  nn.MaxPool2d(2,2)
        self.bn = nn.BatchNorm2d(32)
        self.out = nn.Linear(800,10)
        
    def forward(self,x):
        x = self.pool(self.l1(x))
        x = self.pool(self.l2(x))
        x = self.bn(x)
        x = x.view(4,-1)
        x = self.out(x)
        return x
    
class Net_Ord(nn.Module):
    def __init__(self):
        super(Net_Ord,self).__init__()
        self.l1 = Depth_sc(3,16,3)
        self.l2 = Depth_sc(16,32,5)
        self.pool =  nn.MaxPool2d(2,2)
        self.bn = nn.BatchNorm2d(32)
        self.out = nn.Linear(800,10)
        
    def forward(self,x):
        x = self.pool(self.l1(x))
        x = self.pool(self.l2(x))
        x = self.bn(x)
        x = x.view(4,-1)
        x = self.out(x)
        return x

In [8]:
net = Net_SC()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [9]:
for epoch in range(3):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        #1/0
        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
            
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

[1,  2000] loss: 1.812
[1,  4000] loss: 1.536
[1,  6000] loss: 1.445
[1,  8000] loss: 1.388
[1, 10000] loss: 1.358
[1, 12000] loss: 1.311
[2,  2000] loss: 1.222
[2,  4000] loss: 1.237
[2,  6000] loss: 1.247
[2,  8000] loss: 1.210
[2, 10000] loss: 1.208
[2, 12000] loss: 1.210
[3,  2000] loss: 1.132
[3,  4000] loss: 1.156
[3,  6000] loss: 1.134
[3,  8000] loss: 1.124
[3, 10000] loss: 1.123
[3, 12000] loss: 1.118


In [10]:
net = Net_Ord()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [11]:
for epoch in range(3):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        #1/0
        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
            
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

[1,  2000] loss: 1.873
[1,  4000] loss: 1.602
[1,  6000] loss: 1.522
[1,  8000] loss: 1.465
[1, 10000] loss: 1.461
[1, 12000] loss: 1.402
[2,  2000] loss: 1.350
[2,  4000] loss: 1.352
[2,  6000] loss: 1.315
[2,  8000] loss: 1.290
[2, 10000] loss: 1.283
[2, 12000] loss: 1.263
[3,  2000] loss: 1.237
[3,  4000] loss: 1.208
[3,  6000] loss: 1.180
[3,  8000] loss: 1.177
[3, 10000] loss: 1.176
[3, 12000] loss: 1.180
